# PYNQ tutorial: using GPIO and MMIO


## Contents

* [Goal](#Goal)


* [Tutorial overlay](#Tutorial-overlay)


* [MMIO class](#MMIO-class)


* [GPIO class](#GPIO-class)


## Goal

The aim of this notebook is to show how to use the GPIO and MMIO classes. This example uses the tutorial overlay on PYNQ-Z1 board. 


## Tutorial overlay

The tutorial overlay has been created to allow testing of the PYNQ interface classes, GPIO, MMIO, Xlnk, and DMA. In the tutorial overlay, there is a GPIO controller in the PL which is connected to 2 LEDs, and 2 push buttons on the PYNQ-Z1. GPIO from the Zynq PS are also connected to other 2 LEDs and 2 push buttons. There are also two IOPs, and DMA controllers connected by a FIFO in loopback mode. Only the PS and PL GPIO will be used in this example. 

The GPIO controller in the PL is a memory mapped IP block, and will be accessed from the PYNQ MMIO class. 

The PS GPIO controller pins are also connected to 2 LEDs and 2 buttons through the EMIO, and will be access through the PYNQ GPIO class.

### 1. Download the tutorial overlay

The `pynqtutorial.bit` and `pynqtutorial.tcl` must be in the ``pynq/overlays/pynqtutorial`` folder before the next cell can be executed.

Another way to download the overlay is to instantiate the PynqTutorialOverlay class:
```python
from pynq.overlays.pynqtutorial import PynqTutorialOverlay
overlay = PynqTutorialOverlay("pynqtutorial.bit")
```

In [ ]:
from pynq import Overlay
overlay = Overlay("./bitstream/pynq_tutorial.bit")
overlay.download()

### 2. Check overlay status
Check the IP dictionary of the overlay. This dictionary will show all the IPs available on the bitstream.

In [ ]:
from pprint import pprint

help(overlay)

## GPIO class

The GPIO class will be used to access the GPIO connected to the PL via the EMIO. 

### 1. Controlling the LEDs

In the tutorial overlay, EMIO GPIO pins 7 and 8 are connected to on-board LED 2 (LD2) and LED 3 (LD3). GPIO pins need a lookup to determine the Linux number of the GPIO pin. This is done with ``get_gpio_pin()`` from the GPIO class. Note that the next cell should be executed once only, or the following cells will give an error when executed. 

In [ ]:
from pynq import GPIO
led2 = GPIO(GPIO.get_gpio_pin(7), 'out')
led3 = GPIO(GPIO.get_gpio_pin(8), 'out')

The following cell will turn off LD2, and turn on LD3.

In [ ]:
led2.write(0)
led3.write(1)

The following cell will turn on LD2, and turn off LD3.

In [ ]:
led2.write(1)
led3.write(0)

The following cell will turn off both LD2 and LD3.

In [ ]:
led2.write(0)
led3.write(0)

### 2. Reading the push buttons
Push button 2 (BTN2) and push button 3 (BTN3) on the board are connected to EMIO GPIO 0 and 1. Using a similar approach as before, we can access the on-board buttons on PYNQ-Z1.

In [ ]:
button2 = GPIO(GPIO.get_gpio_pin(0), 'in')
button3 = GPIO(GPIO.get_gpio_pin(1), 'in')

Now press BTN2 or BTN3 and run the following cell. 

In [ ]:
print(f'Push button 2 reads {button2.read()}.')
print(f'Push button 3 reads {button3.read()}.')

You can rerun the cell above with different combinations of the two buttons pressed.

## MMIO class

The PL GPIO controller has two channels. Channel 1 is mapped to two push buttons on the PYNQ-Z1. Channel 2 is mapped to two LEDs on the board. Channel 1 is mapped to register offset 0 in the GPIO instance, and channel 2 is mapped to register offset 0x8. 

MMIO can map arrays, or a range of addresses. 

A single MMIO instance with the following conditions is able to access the data for both channels of the GPIO controller:
* Starting at the base address of the GPIO controller.
* Covering the address range of 3 consecutive registers (register 0x0, 0x4, and 0x8).

Note that the second register 0x4 will be mapped to the same MMIO, but this is not mandatory. 

In the following example, 2 MMIO instances will be created, each mapped to a single register: channel 1 (0x0) and channel 2 (0x8).

### 1. Reading the push buttons

The two push buttons (BTN0 and BTN1) are connected to PL GPIO controller and will be accessed from the MMIO class. They are mapped to register at address offset 0, and can be accessed by reading the base address of the PL GPIO controller.

An instance of the MMIO is created, called `buttons_mmio`; the base address of the PL GPIO controller will be mapped to this MMIO instance.

In [ ]:
button_address = overlay.ip_dict['btns_gpio']['phys_addr']
print("Physical address of button: 0x" + format(button_address, '02x'))

In [ ]:
from pynq import MMIO
buttons = MMIO(button_address) 

We use the following cell as an example to read the push button values; this value will be a 2-bit value read from BTN0 and BTN1:
* If neither button is pressed, the value will be 0.
* If only push button 0 is pressed, the value will be 1.
* If only push button 1 is pressed, the value will be 2.
* If both buttons are pressed, the value will be 3.

In [ ]:
buttons.read()

### 2. Controlling the LEDs
In the next cell, let us repeat the process for LEDs.

In [ ]:
led_address = overlay.ip_dict['swsleds_gpio']['phys_addr']
print("Physical address of LED: 0x" + format(led_address, '02x'))

The address offset for channel 2 of the GPIO is 0x8.

In [ ]:
CH2_OFFSET = 0x8
led_address = overlay.ip_dict['swsleds_gpio']['phys_addr']
leds = MMIO(led_address + CH2_OFFSET)

The following cells will write to register 0x8. The lower two bits will be written to LEDs LD1 and LD0.

Let's turn on LD0 only.

In [ ]:
leds.write(0, 1)

Now turn on LD1 only.

In [ ]:
leds.write(0, 2)

Turn both LD0 and LD1 off.

In [ ]:
leds.write(0, 0)